# Morphing water vapor from SAPHIR
### Erfan Jahangir and Brian Mapes March 2017

#### General strategy of this notebook

Like any programming job, we work backward from a result-shaped container, and forward from the most precious original data input whose value we are trying to optimize. These intentions make it natural to build the middle steps and grab auxiliary inputs, as necessary to achieve success. Initially we achieve success in the simplest way, then later add sophistication -- but only to the extent it improves signal and reduces noise, as seen in the initial success of the data products. 

In this case, the results-shaped container is a single 1/2-degree (for now) array of 

1. **WV** (water vapor)
1. **AT** (the 'analysis tendency' of the morphing operation, representing all the physical source-sink terms). 
1. **t_early** (the time of the earlier observation that made the above products)
1. **t_late** (the time of the later observation " " " )

In this case, the precious data is swaths of data, made up of pixels, of WV500 retrieved from SAPHIR channels by Helene Brignietz and colleagues. These input data include arrays of: 

* the WV **value** at each pixel
* the **latitude** " " " 
* the **longitude** " " " 
* the **time of observation** " " " 

----------

In [35]:
import numpy as np 

### Step 1: Create the results-shaped containers  

In [39]:
DX = 0.5 # degrees
SOUTH = -30 # Central latitudes of southern grid  cells
NORTH = 30  # " " northern " " 
WEST = 0.25 # Central longitude of westernmost cell
EAST = 359.75 # " " easternmost " 

# Build 1D spatial coordinate arrays 
NLAT = int( (NORTH-SOUTH)/DX )
lat = np.linspace(SOUTH, NORTH, NLAT)

NLON = int( (EAST-WEST)/DX )
lon = np.linspace(WEST, EAST, NLON)


# Now build containers for the results we desire
# Which order? LON,LAT? or LAT,LON? 

WV = np.zeros( (NLON,NLAT) )
AT = np.zeros( (NLON,NLAT) )

In [40]:
whos

Variable    Type       Data/Info
--------------------------------
AT          ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
AT500       ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
DX          float      0.5
EAST        float      359.75
NLAT        int        120
NLON        int        719
NORTH       int        30
SLAT        int        -30
SOUTH       int        -30
WEST        float      0.25
WV          ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV500       ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
dt_after    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
dt_before   ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
lat         ndarray    120: 120 elems, type `float64`, 960 bytes
lon         ndarray    719: 719 elems, type `float64`, 5752 bytes
np          module     <module 'numpy' from '//a<...>kag

### Step 2: The things we need to fill the containers

To fill the above arrays, we will use the *time-proximity-weighted average* for WV500:  

$ WV500 = ( WV_{before}*dt_{after} + WV_{before}*dt_{after} )/(dt_{before} + dt_{after}) $ 

and the simplest *estimate of the time derivative* using the before and after observations:

$ AT500 = ( WV_{after} - WV_{before})/(dt_{before} + dt_{after}) $ 

**Thus, we need $ WV_{before}, WV_{after}, dt_{before}, dt_{after}  $**

----------------

In [43]:
# Containers for the necessary quantities to get our final products

WV_before = np.zeros( (NLON,NLAT) )
WV_after  = np.zeros( (NLON,NLAT) )
dt_before = np.zeros( (NLON,NLAT) )
dt_after  = np.zeros( (NLON,NLAT) )

In [44]:
whos

Variable    Type       Data/Info
--------------------------------
AT          ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
AT500       ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
DX          float      0.5
EAST        float      359.75
NLAT        int        120
NLON        int        719
NORTH       int        30
SLAT        int        -30
SOUTH       int        -30
WEST        float      0.25
WV          ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV500       ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_after    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_before   ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
dt_after    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
dt_before   ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
lat        

## Now, how to get $ WV_{before}, WV_{after}, dt_{before}, dt_{after} $ ? 

#### We need two _time stacks_ of product-shaped lat-lon arrays. 

Two, because we need the VALUE and the TIME of each observation we put into the stack. 

These stack is a 3D array, centered on the product time (dt=0). It doesn't matter how long in time  this stack extends, as long as it is long enough that **every pixel in space has a before and an after observation**. That is, the DTMAX just has to be at least as big as the longest time gap between observations. Also, the time step between the layers in the stack just has to be short enough that we aren't wasting observations by over-writing some locations with multiple observations. Since the orbit time is about 100 minutes, 1 hour stacks are safe. 

Initially, let's make a container full of zeros. 

In [45]:
DTMAX = 6    # hours. The size of the centered stack will then be 2*DTMAX+1. 
DTstack = 1  # hour

WV_stack = np.zeros( (NLON,NLAT, 2*DTMAX+1) )
tobs_stack = np.zeros( (NLON,NLAT, 2*DTMAX+1) )

In [46]:
whos

Variable     Type       Data/Info
---------------------------------
AT           ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
AT500        ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
DTMAX        int        6
DTstack      int        1
DX           float      0.5
EAST         float      359.75
NLAT         int        120
NLON         int        719
NORTH        int        30
SLAT         int        -30
SOUTH        int        -30
WEST         float      0.25
WV           ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV500        ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_after     ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_before    ndarray    719x120: 86280 elems, type `float64`, 690240 bytes (674.0625 kb)
WV_stack     ndarray    719x120x13: 1121640 elems, type `float64`, 8973120 bytes (8.55743408203125 Mb)
dt_after     nda